In [8]:
import pandas as pd

orders = pd.read_csv('orders.csv')
orders["Timestamp"] = pd.to_datetime(orders["Timestamp"])
orders.head()

,Timestamp,Group ID,Side,Price,Size
0,2024-09-30 20:26:01,5,buy,8.0,50
1,2024-09-30 20:26:11,14,sell,11.0,20
2,2024-09-30 20:26:14,10,sell,11.0,100
3,2024-09-30 20:26:15,12,sell,10.0,50
4,2024-09-30 20:26:16,7,buy,5.0,50


## Main structure of CLOB
- deal with each order and try to match a successful trade
- place in a proper position if not matched / partly matched

In [9]:
# rename to bid & ask in the final display
clob = {"buy": {}, "sell": {}}
trade = pd.DataFrame(columns=["Timestamp", "buy", "sell", "Price", "Volume"]) 

In [10]:
def match_trade(order):
    side = order["Side"]
    pass

In [11]:
# Loop through all trades

for i in range(0, len(orders)):
    order = orders.iloc[i]
    if match_trade:
        # Place in the Trade Records
        pass
    else:
        # Place in the Order Book
        pass

